In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
## Imports and environment variables 
import os
import torch
import wandb
from travis_attack.utils import set_seed, set_session_options, setup_logging, setup_parser, resume_wandb_run, display_all
from travis_attack.config import Config
from travis_attack.models import prepare_models, get_optimizer
from travis_attack.data import ProcessedDataset
from travis_attack.trainer import Trainer
from travis_attack.insights import (postprocess_df, create_and_log_wandb_postrun_plots, get_training_dfs)
from fastcore.basics import in_jupyter

import logging 
logger = logging.getLogger("run")

In [ ]:
from nbdev.export import notebook2script
notebook2script()

!jupyter nbconvert \
    --TagRemovePreprocessor.enabled=True \
    --TagRemovePreprocessor.remove_cell_tags="['hide']" \
    --TemplateExporter.exclude_markdown=True \
    --to python "run.ipynb"

Converted 00_utils.ipynb.
Converted 02_tests.ipynb.
Converted 03_config.ipynb.
Converted 07_models.ipynb.
Converted 10_data.ipynb.
Converted 20_trainer.ipynb.
Converted 25_insights.ipynb.
Converted baselines.ipynb.
Converted baselines_analysis.ipynb.
Converted index.ipynb.
Converted run.ipynb.
Converted show_examples.ipynb.
Converted test_pp_model.ipynb.
[NbConvertApp] Converting notebook run.ipynb to python


In [ ]:
cfg = Config()  # default values
if not in_jupyter():  # override with any -- options when running with command line
    parser = setup_parser()
    newargs = vars(parser.parse_args())
    for k,v in newargs.items(): 
        if v is not None: setattr(cfg, k, v)
if cfg.use_small_ds:  cfg = cfg.small_ds()
set_seed(cfg.seed)
set_session_options()
setup_logging(cfg, disable_other_loggers=True)
vm_tokenizer, vm_model, pp_tokenizer, pp_model, ref_pp_model, sts_model, nli_tokenizer, nli_model, cfg = prepare_models(cfg)
optimizer = get_optimizer(cfg, pp_model)

In [ ]:
ds = ProcessedDataset(cfg, vm_tokenizer, vm_model, pp_tokenizer, sts_model, load_processed_from_file=False)

travis_attack.data: INFO     Will load dataset rotten_tomatoes with use_small_ds set to False
travis_attack.data: INFO     Will load dataset rotten_tomatoes with use_small_ds set to False
travis_attack.data: INFO     Will load dataset rotten_tomatoes with use_small_ds set to False


In [ ]:
ds.dsd_raw

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'idx', 'text_with_prefix'],
        num_rows: 7443
    })
    test: Dataset({
        features: ['text', 'label', 'idx', 'text_with_prefix'],
        num_rows: 889
    })
    valid: Dataset({
        features: ['text', 'label', 'idx', 'text_with_prefix'],
        num_rows: 895
    })
})

In [ ]:
ds.show_random_elements(ds.dsd_raw['train'])

,text,label,idx,text_with_prefix
0,compellingly watchable .,pos,555,paraphrase: compellingly watchable . </s>
1,"while obviously an extremely personal work , it remains inextricably stuck in an emotionally unavailable rut .",neg,7091,"paraphrase: while obviously an extremely personal work , it remains inextricably stuck in an emotionally unavailable rut . </s>"
2,the quirky and recessive charms of co-stars martin donovan and mary-louise parker help overcome the problematic script .,pos,2256,paraphrase: the quirky and recessive charms of co-stars martin donovan and mary-louise parker help overcome the problematic script . </s>
3,a sophomoric exploration of 'life problems' most people solved long ago -- or at least got tired of hearing people kvetch about .,neg,7171,paraphrase: a sophomoric exploration of 'life problems' most people solved long ago -- or at least got tired of hearing people kvetch about . </s>
4,"the movie is brilliant , really . it is philosophy , illustrated through everyday events .",pos,2490,"paraphrase: the movie is brilliant , really . it is philosophy , illustrated through everyday events . </s>"
5,"a rich tale of our times , very well told with an appropriate minimum of means .",pos,2078,"paraphrase: a rich tale of our times , very well told with an appropriate minimum of means . </s>"
6,"the actors improvise and scream their way around this movie directionless , lacking any of the rollicking dark humor so necessary to make this kind of idea work on screen .",neg,8106,"paraphrase: the actors improvise and scream their way around this movie directionless , lacking any of the rollicking dark humor so necessary to make this kind of idea work on screen . </s>"
7,"why make a documentary about these marginal historical figures ? wouldn't one about their famous dad , author of death in venice , etc . , be more valuable ?",neg,5822,"paraphrase: why make a documentary about these marginal historical figures ? wouldn't one about their famous dad , author of death in venice , etc . , be more valuable ? </s>"
8,"a bit of a downer and a little over-dramatic at times , but this is a beautiful film for people who like their romances to have that french realism .",pos,2408,"paraphrase: a bit of a downer and a little over-dramatic at times , but this is a beautiful film for people who like their romances to have that french realism . </s>"
9,it's just a little too self-satisfied .,neg,4729,paraphrase: it's just a little too self-satisfied . </s>


In [ ]:

# def _get_ref_logprobs(orig_ids, pp_ids): 
#     orig_batch_size = len(orig_ids)
#     decoder_start_token_ids = torch.tensor([ref_pp_model.config.decoder_start_token_id], device=cfg.device).repeat(orig_batch_size, 1)
#     pp_ids = torch.cat([decoder_start_token_ids, pp_ids], 1)
#     logprobs = []
#     with torch.no_grad(): 
#         for i in range(pp_ids.shape[1] - 1): 
#             decoder_input_ids = pp_ids[:, 0:(i+1)]
#             outputs = ref_pp_model(input_ids=orig_ids, decoder_input_ids=decoder_input_ids)
#             token_logprobs = outputs.logits[:,i,:].log_softmax(1)
#             pp_next_token_ids = pp_ids[:,i+1].unsqueeze(-1)
#             pp_next_token_logprobs = torch.gather(token_logprobs, 1, pp_next_token_ids).detach().squeeze(-1)
#             logprobs.append(pp_next_token_logprobs)
#     logprobs = torch.stack(logprobs, 1)   
#     attention_mask = ref_pp_model._prepare_attention_mask_for_generation(pp_ids[:,1:],
#             pp_tokenizer.pad_token_id, pp_tokenizer.eos_token_id)
#     logprobs = logprobs * attention_mask        
#     logprobs_sum = logprobs.sum(1)
#     logprobs_normalised = logprobs_sum / attention_mask.sum(1)  # normalise for length of generated sequence
#     return logprobs_normalised

# orig_l = [
#     "the actors pull out all the stops in nearly every scene , but to diminishing effect . the characters never change . <s>",
#     "compassionately explores the seemingly irreconcilable situation between conservative christian parents and their estranged gay and lesbian children .",
#     "apart from anything else , this is one of the best-sustained ideas i have ever seen on the screen .",
#     "my name is Tom and I like driving cars"
# ]

# pp_l = [
#     "The actors pull out everything they touch in a scene but that it will diminish",
#     "There is a situation between two families of separated gay and lesbian children.",
#     "I have seen many of the similar ideas on screen but the one that has lasted most was by this",
#     "my name is Tom and I like to drive fast cars"
# ]
# orig_ids = pp_tokenizer(orig_l, return_tensors='pt', padding=True, truncation=True).input_ids.to(cfg.device)
# pp_ids   = pp_tokenizer(pp_l,   return_tensors='pt', padding=True, truncation=True).input_ids.to(cfg.device)

# %lprun -f _get_ref_logprobs logprobs_normalised = _get_ref_logprobs(orig_ids, pp_ids)

In [ ]:
cfg.wandb['mode'] = 'disabled'
trainer = Trainer(cfg, vm_tokenizer, vm_model, pp_tokenizer, pp_model, ref_pp_model, sts_model, nli_tokenizer, nli_model, optimizer,
                  ds, initial_eval=False, use_cpu=False)
print(vars(cfg))
trainer.train()

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find run.


{'pp_name': 'prithivida/parrot_paraphraser_on_T5', 'dataset_name': 'simple', 'sts_name': 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', 'nli_name': 'howey/electra-small-mnli', 'vm_name': 'textattack/distilbert-base-uncased-rotten-tomatoes', 'seed': 420, 'use_small_ds': False, 'sampling_strategy': 'sample', 'lr': 1e-05, 'kl_coef': 0.01, 'reward_fn': 'reward_fn_contradiction_and_letter_diff', 'pin_memory': True, 'zero_grad_with_none': False, 'pad_token_embeddings': False, 'embedding_padding_multiple': 8, 'orig_padding_multiple': 8, 'bucket_by_length': True, 'shuffle_train': False, 'remove_misclassified_examples': True, 'unfreeze_last_n_layers': 'all', 'pp': {'num_return_sequences': 1, 'temperature': 1, 'length_penalty': 1, 'min_length': 5, 'max_length': 20, 'do_sample': True, 'num_beams': 1}, 'n_shards': None, 'shard_contiguous': None, 'save_model_while_training': False, 'save_model_freq': 10, 'wandb': {'project': 'travis_attack', 'entity': 'uts_nlp', 'mode': 'disabled', 

travis_attack.trainer: INFO     Now on epoch 1 of 50
travis_attack.trainer: INFO     Now on epoch 1 of 50
travis_attack.trainer: INFO     Now on epoch 1 of 50


travis_attack.trainer: INFO     Now on epoch 2 of 50
travis_attack.trainer: INFO     Now on epoch 2 of 50
travis_attack.trainer: INFO     Now on epoch 2 of 50


travis_attack.trainer: INFO     Now on epoch 3 of 50
travis_attack.trainer: INFO     Now on epoch 3 of 50
travis_attack.trainer: INFO     Now on epoch 3 of 50


...

travis_attack.trainer: INFO     Now on epoch 46 of 50
travis_attack.trainer: INFO     Now on epoch 46 of 50
travis_attack.trainer: INFO     Now on epoch 46 of 50


travis_attack.trainer: INFO     Now on epoch 47 of 50
travis_attack.trainer: INFO     Now on epoch 47 of 50
travis_attack.trainer: INFO     Now on epoch 47 of 50


travis_attack.trainer: INFO     Now on epoch 48 of 50
travis_attack.trainer: INFO     Now on epoch 48 of 50
travis_attack.trainer: INFO     Now on epoch 48 of 50


travis_attack.trainer: INFO     Now on epoch 49 of 50
travis_attack.trainer: INFO     Now on epoch 49 of 50
travis_attack.trainer: INFO     Now on epoch 49 of 50


travis_attack.trainer: INFO     Now on epoch 50 of 50
travis_attack.trainer: INFO     Now on epoch 50 of 50
travis_attack.trainer: INFO     Now on epoch 50 of 50


In [ ]:
# ## TO RESUME RUN
# cfg = Config()
# cfg.run_id = '2jq83fdx'
# cfg.run_name = "pleasant-wind-125"
# cfg.path_run = f"{cfg.path_checkpoints}{cfg.run_name}/"
# run = resume_wandb_run(cfg)


In [ ]:
df_d = get_training_dfs(cfg.path_run, postprocessed=False)
for k, df in df_d.items(): 
    df_d[k] = postprocess_df(df, filter_idx=None, num_proc=1)
    df_d[k].to_pickle(f"{cfg.path_run}{k}_postprocessed.pkl")    
create_and_log_wandb_postrun_plots(df_d)
trainer.run.finish()
#run.finish()